Import stuff like in AlexaNet

In [ ]:

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Lambda, AveragePooling2D, Activation, BatchNormalization, Softmax
from tensorflow.keras.layers.experimental.preprocessing import Normalization, Rescaling
 


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')                         #mount google drive for the dataset

#DeepDocClassifier Neural Network

Here's where the fun begins


In [ ]:
#Trainingset of 100
DATADIR = r'/content/drive/MyDrive/Colab Notebooks/Traindata/'

In [ ]:
#Trainingset of 80
DATADIR = r'/content/drive/MyDrive/Colab Notebooks/Traindata ALL/Traindata80'

In [ ]:
#Trainingset of 60
DATADIR = r'/content/drive/MyDrive/Colab Notebooks/Traindata ALL/Traindata60'

In [ ]:
#Trainingset of 40
DATADIR = r'/content/drive/MyDrive/Colab Notebooks/Traindata ALL/Traindata60'

In [ ]:
#Trainingset of 20
DATADIR = r'/content/drive/MyDrive/Colab Notebooks/Traindata ALL/Traindata20'

In [ ]:

batch_size = 10
IMG_SIZE = 227
CLASS_NAMES = ['ADVE', 'Email', 'Form', 'Letter', 'Memo', 'News', 'Note', 'Report', 'Resume', 'Scientific']


In [ ]:
#create Trainingdataset and Split 20% for Validation
train_ds = tf.keras.preprocessing.image_dataset_from_directory(     
  DATADIR,
  #label_mode= 'categorical',
  class_names =CLASS_NAMES,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size,
)


In [ ]:
#create Validationsubset with thesame seed
val_ds = tf.keras.preprocessing.image_dataset_from_directory(     
  DATADIR,
  #label_mode= 'categorical',
  class_names =CLASS_NAMES,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_SIZE,IMG_SIZE),
  batch_size=batch_size,
  )


In [ ]:
CL = train_ds.class_names

In [ ]:
print (CL)

In [ ]:
#plot some traindatas to test if the data is correctly implemented

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(CL[labels[i]])
    plt.axis("off")



In [ ]:
#Weights from AlexNet that I wasn't able to insert in my NN
net_data = np.load('/content/drive/MyDrive/Colab Notebooks/AlexNet Implementation/bvlc_alexnet.npy', encoding='bytes', allow_pickle=True).item()

In [ ]:
#print (tf.Variable(net_data['conv1']))

In [ ]:
#Weights need to be edit

In [ ]:
#My Concolutional Neural Network with 8 Layers

i_s = (227,227,3)
k='he_normal'

model = Sequential()

#scale pixels for values between [0,1]
model.add(Rescaling(1./255, input_shape = i_s))

#preprocess Images with mean and std from AlexNet
model.add(Normalization(mean=[0.485, 0.456, 0.406], variance =[0.229, 0.224, 0.225]))


#Conv1
model.add(Conv2D(96, (11,11), input_shape=i_s, strides=4, activation = 'relu', kernel_initializer= k))
model.add(BatchNormalization())
#model.layers[0].weights = (tf.Variable(net_data['conv1'])) #this doesn't work at all
#model.add(Activation(tf.nn.local_response_normalization))
#Maxpool1
model.add(MaxPooling2D(pool_size=(3, 3),strides=2))


#Conv2
model.add(Conv2D(256, (5,5), strides = 1, activation = 'relu', padding='same', kernel_initializer= k))
model.add(BatchNormalization())
#model.add(Activation(tf.nn.local_response_normalization))
#Maxpool2
model.add(MaxPooling2D(pool_size=(3,3),strides=2, padding='valid', data_format= None))


#Conv3
model.add(Conv2D(384, kernel_size=(3,3), strides= 1, padding= 'same', activation= 'relu', kernel_initializer= k))
model.add(BatchNormalization())
#Conv4
model.add(BatchNormalization())
model.add(Conv2D(384, kernel_size=(3,3), strides=1, padding= 'same', activation= 'relu', kernel_initializer= k))


#Conv5
model.add(Conv2D(256, kernel_size=(3,3), strides=1, padding= 'same', activation= 'relu', kernel_initializer= k))
model.add(BatchNormalization())
#Maxpool3
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3),strides=2, padding='valid', data_format= None))


#convert our 3D feature maps to 1D feature vectors
model.add(Flatten())
#FC6
model.add(Dense(4096, activation='relu'))
#Dropout1 to prevent overfitting
model.add(Dropout(rate = 0.5))


#FC7
model.add(Dense(4096, activation='relu'))
#Dropout2 
model.add(Dropout(rate =0.5 ))


#FC8
model.add(Dense(10, activation= 'softmax'))
#model.add(Dense(10, activation= 'softmax'))


#compile model with Stochastic Gradient Descent
model.compile(optimizer= tf.keras.optimizers.SGD(learning_rate =0.0001, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#train the model
history = model.fit(train_ds, validation_data = val_ds, epochs=50)

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/saved model/Epoch80')
#model.save('/content/drive/MyDrive/Colab Notebooks/saved model/Epoch60')
#model.save('/content/drive/MyDrive/Colab Notebooks/saved model/Epoch40')
#model.save('/content/drive/MyDrive/Colab Notebooks/saved model/Epoch20')


In [ ]:
model.summary()

In [ ]:
history.history

In [ ]:
#define variables for the plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
#plot Training Accuracy and Validation Accuracy for the Epochs

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()